## 테스트 및 Django 사용

In [4]:
import pandas as pd
import numpy as np
import pickle  

In [5]:
# load
with open('./ratings_matrix.bin', 'rb') as f:
    ratings_matrix = pickle.load(f)

print(type(ratings_matrix))    
print(ratings_matrix.shape)    
display(ratings_matrix[0:3]) 

<class 'pandas.core.frame.DataFrame'>
(610, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# load
with open('./ratings_pred_matrix.bin', 'rb') as f:
    ratings_pred_matrix = pickle.load(f)
    
print(type(ratings_pred_matrix))    
print(ratings_pred_matrix.shape)    
display(ratings_pred_matrix[0:3])   

<class 'pandas.core.frame.DataFrame'>
(610, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.220798,0.0,0.0,1.677291,0.284372
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


In [7]:
# 사용자가 관람하지 않은 영화에 대해서만 추천을 수행하기위해
# 관람하지 않은 영화명 추출
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화 정보를 추출해 Series 로 반환함
    # 반환된 user_rating 은 영화명(title)을 인덱스로 가지는 Series 객체임
    user_rating = ratings_matrix.loc[userId, :]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 인덱스를 추출해 list 객체로 만듦.
    already_seen = user_rating[user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듦.
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension 으로 already_seen에 해당하는 영화는 movie_list에서 제외함.
    unseen_list = [movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

In [8]:
# 특정 사용자에게 영화를 추천
# pred_df: 평점 예측 데이터, userId: 사용자 id, unseen_list: 시청하지 않은 index list, top_n=10: 추천할 영화수
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출해
    # 가장 예측 평점이 높은 순으로 정렬함
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [9]:
# 9번 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, userId=9)

# 아이템 기반의 최근접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

recom = recomm_movies.values
print(type(recom))
display(recom)
display(recomm_movies.index)

# 평점 데이터를 DataFrame 으로 생성, ndarray -> DataFrame 생성, index=recomm_movies.index
# columns=['pred_score']: 컬럼명 새로 지정
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index
                            ,columns=['pred_score'])
display(recomm_movies)

<class 'numpy.ndarray'>


array([0.86620187, 0.8578536 , 0.81747328, 0.81662608, 0.80099031,
       0.76515861, 0.74095567, 0.73269329, 0.68959056, 0.67671083])

Index(['Shrek (2001)', 'Spider-Man (2002)', 'Last Samurai, The (2003)',
       'Indiana Jones and the Temple of Doom (1984)',
       'Matrix Reloaded, The (2003)',
       'Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)',
       'Gladiator (2000)', 'Matrix, The (1999)',
       'Pirates of the Caribbean: The Curse of the Black Pearl (2003)',
       'Lord of the Rings: The Return of the King, The (2003)'],
      dtype='object', name='title')

,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591


## Django

In [15]:
import pandas as pd
import numpy as np
import pickle

def recommend2(memberno):
    # load
    with open('./ratings_matrix.bin', 'rb') as f:
        ratings_matrix = pickle.load(f)

    with open('./ratings_pred_matrix.bin', 'rb') as f:
        ratings_pred_matrix = pickle.load(f)

    # memberno 사용자가 관람하지 않은 영화명 추출
    unseen_list = get_unseen_movies(ratings_matrix, userId=memberno)

    # 아이템 기반의 최근접 이웃 협업 필터링으로 영화 추천
    recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, memberno, unseen_list, top_n=10)
    recom = recomm_movies.values

    # 평점 데이터를 DataFrame 으로 생성, ndarray -> DataFrame 생성, index=recomm_movies.index
    # columns=['pred_score']: 컬럼명 새로 지정
    recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index
                                ,columns=['pred_score'])
    return recomm_movies

# 사용자가 관람하지 않은 영화에 대해서만 추천을 수행하기위해
# 관람하지 않은 영화명 추출
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화 정보를 추출해 Series 로 반환함
    # 반환된 user_rating 은 영화명(title)을 인덱스로 가지는 Series 객체임
    user_rating = ratings_matrix.loc[userId, :]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 인덱스를 추출해 list 객체로 만듦.
    already_seen = user_rating[user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듦.
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension 으로 already_seen에 해당하는 영화는 movie_list에서 제외함.
    unseen_list = [movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

# 특정 사용자에게 영화를 추천
# pred_df: 평점 예측 데이터, userId: 사용자 id, unseen_list: 시청하지 않은 index list, top_n=10: 추천할 영화수
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=5):
    # 예측 평점 DataFrame에서 사용자id 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출해
    # 가장 예측 평점이 높은 순으로 정렬함
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [16]:
# 회원 추천 상품 update
membernos = [1, 2, 3, 4, 5]
for memberno in membernos:
    print('회원 번호:', memberno)
    display(recommend2(memberno))    

회원 번호: 1


,pred_score
title,
"Godfather: Part II, The (1974)",3.769505
"Sixth Sense, The (1999)",3.689380
"Godfather, The (1972)",3.503021
Trainspotting (1996),3.460017
Die Hard (1988),3.435728
Ferris Bueller's Day Off (1986),3.396243
Blade Runner (1982),3.227809
Airplane! (1980),3.200097
Jaws (1975),3.170789


회원 번호: 2


,pred_score
title,
"Social Network, The (2010)",1.381515
The Imitation Game (2014),1.345850
Gran Torino (2008),1.162998
"Grand Budapest Hotel, The (2014)",1.128498
District 9 (2009),0.970748
The Revenant (2015),0.969021
Gone Girl (2014),0.968061
"Hangover, The (2009)",0.922380
Fight Club (1999),0.903515


회원 번호: 3


,pred_score
title,
Mad Max (1979),0.479083
Mad Max Beyond Thunderdome (1985),0.464510
Big Trouble in Little China (1986),0.446975
Escape from New York (1981),0.379657
Star Trek III: The Search for Spock (1984),0.332750
Logan's Run (1976),0.328926
Star Trek IV: The Voyage Home (1986),0.297457
Superman II (1980),0.295319
Godzilla (Gojira) (1954),0.294782


회원 번호: 4


,pred_score
title,
Ferris Bueller's Day Off (1986),2.493093
Raising Arizona (1987),2.085108
Office Space (1999),2.012212
Citizen Kane (1941),1.841979
To Kill a Mockingbird (1962),1.750454
"Usual Suspects, The (1995)",1.748657
"Godfather: Part II, The (1974)",1.745427
Trainspotting (1996),1.743102
"Godfather, The (1972)",1.732375


회원 번호: 5


,pred_score
title,
Forrest Gump (1994),2.220658
Jurassic Park (1993),2.049475
Star Trek: Generations (1994),1.917485
Mrs. Doubtfire (1993),1.797205
"Silence of the Lambs, The (1991)",1.782353
Waterworld (1995),1.757004
Dave (1993),1.709318
Die Hard: With a Vengeance (1995),1.630076
Twelve Monkeys (a.k.a. 12 Monkeys) (1995),1.595411
